<a href="https://colab.research.google.com/github/diyism/colab_kaldi2/blob/main/my_icefall_wenetspeech_asr_dataset_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls -al /content/icefall/egs/wenetspeech/ASR

total 348
drwxr-xr-x 9 root root   4096 Oct  4 18:59 .
drwxr-xr-x 4 root root   4096 Oct  4 16:43 ..
-rw------- 1 root root 224044 Oct  4 18:56 4_me.wav
drwxr-xr-x 4 root root   4096 Oct  4 18:56 data
drwxr-xr-x 3 root root   4096 Oct  4 17:21 download
-rwxr-xr-x 1 root root   2651 Oct  4 16:43 finetune.sh
-rw------- 1 root root  41644 Oct  4 18:56 jiang3you3bo2.wav
drwxr-xr-x 2 root root   4096 Oct  4 17:43 local
-rw-r--r-- 1 root root     99 Oct  4 17:29 log-preprocess-wenetspeech-2024-10-04-17-29-45
-rwxrwxrwx 1 root root   1912 Oct  4 18:59 prepare.sh
-rwxr-xr-x 1 root root  13760 Oct  4 16:43 prepare.sh.bak
-rwxrwxrwx 1 root root   1785 Oct  4 16:45 prepare.sh.bak1
drwxr-xr-x 2 root root   4096 Oct  4 16:43 pruned_transducer_stateless2
drwxr-xr-x 2 root root   4096 Oct  4 16:43 pruned_transducer_stateless5
-rw-r--r-- 1 root root   1041 Oct  4 16:43 README.md
-rw-r--r-- 1 root root  10776 Oct  4 16:43 RESULTS.md
lrwxrwxrwx 1 root root     28 Oct  4 16:43 shared -> ../../librispeech

In [ ]:
#1. install lhotse and lhotse import and cut
#Normally, we would use pip install lhotse. However, the yesno recipe is added recently and has not been released to PyPI yet,
#so we install the latest unreleased version here.
!pip install -q git+https://github.com/lhotse-speech/lhotse

from google.colab import drive
!umount /content/drive
drive.mount('/content/drive', force_remount=True)

#!ls -al /content/drive/MyDrive/ColabData/KWS/kws_create_dataset/train
#!lhotse kaldi import --help
#!cd /content/drive/MyDrive/ColabData/KWS/kws_create_dataset && lhotse kaldi import ./train/ 16000 ./train_manifests/
!ls -al /content/drive/MyDrive/ColabData/KWS/kws_create_dataset/train_manifests

#-f ./train_manifests/features.jsonl.gz \
!cd /content/drive/MyDrive/ColabData/KWS/kws_create_dataset && lhotse cut simple \
  -r ./train_manifests/recordings.jsonl.gz \
  -s ./train_manifests/supervisions.jsonl.gz \
  ./myvoice_train.jsonl.gz

!ls -al /content/drive/MyDrive/ColabData/KWS/kws_create_dataset/

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.3 MB/s eta 0:00:00
umount: /content/drive: no mount point specified.
Mounted at /content/drive
total 2
-rw------- 1 root root 358 Sep 28 15:56 cuts.jsonl.gz
-rw------- 1 root root 209 Sep 28 15:56 recordings.jsonl.gz
-rw------- 1 root root 205 Sep 28 15:56 supervisions.jsonl.gz
total 414
-rw------- 1 root root     58 Sep 26 02:38 4_me.txt
-rw------- 1 root root 224044 Sep 26 02:34 4_me.wav
-rw------- 1 root root     58 Sep 26 03:46 4.txt
-rw------- 1 root root 147216 Sep 26 02:39 4.wav
-rw------- 1 root root     15 Sep 26 02:40 jiang3you3bo2.txt
-rw------- 1 root root  41644 Sep 26 02:40 jiang3you3bo2.wav
-rw------- 1 root root    367 Oct  4 16:42 myvoice_train.jsonl.gz
drwx-----

In [ ]:
# originate from https://k2-fsa.github.io/icefall/recipes/Non-streaming-ASR/yesno/tdnn.html#colab-notebook
#2. Install PyTorch, torchaudio, k2
import torch
print(torch.__version__)

!pip install -q torchaudio
import torchaudio
print(torchaudio.__version__)

#the cuda version and torch version should match the upper printed verions
!pip install -q k2==1.24.4.dev20240905+cuda12.1.torch2.4.1 -f https://k2-fsa.github.io/k2/cuda.html
!python3 -m k2.version

2.4.1+cu121
2.4.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 MB 8.7 MB/s eta 0:00:00

k2 version: 1.24.4
Build type: Release
Git SHA1: cf664841c6d93e21e59b40aade84869b76c919c1
Git date: Thu Sep 5 19:25:17 2024
Cuda used to build k2: 12.1
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.30.2
GCC version: 9.3.1
CMAKE_CUDA_FLAGS: -Wno-deprecated-gpu-targets -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_50,code=sm_50 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_60,code=sm_60 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_61,code=sm_61 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --expt-extended-lambda -gencode arch=compute_70,code=sm_70 -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w --ex

In [ ]:
#3. install icefall
#k2-icefall is a collection of Python scripts.You don't need to install it. What you need to do is to get its source code,
#install its dependencies, and set the PYTHONPATH pointing to it.
!git clone https://github.com/k2-fsa/icefall
!pip install -q --upgrade onnxconverter-common
!cd icefall && pip install -q -r requirements.txt

Cloning into 'icefall'...
remote: Enumerating objects: 18395, done.
remote: Counting objects: 100% (703/703), done.
remote: Compressing objects: 100% (384/384), done.
remote: Total 18395 (delta 373), reused 548 (delta 278), pack-reused 17692 (from 1)
Receiving objects: 100% (18395/18395), 20.29 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (12469/12469), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; pyth

In [ ]:
#4. data preparation

# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
#! pip uninstall -y tensorflow

! cd /content/icefall/egs/wenetspeech/ASR && \
wget https://github.com/diyism/colab_kaldi2/raw/refs/heads/main/icefall_egs_wenetspeech_ASR_prepare.sh -O prepare.sh && \
chmod 777 prepare.sh

! cd /content/icefall/egs/wenetspeech/ASR/local && \
wget https://github.com/diyism/colab_kaldi2/raw/refs/heads/main/icefall_egs_wenetspeech_ASR_local_preprocess_wenetspeech.py -O preprocess_wenetspeech.py

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/wenetspeech/ASR && \
  rm -rf data && \
  ./prepare.sh

--2024-10-04 18:56:19--  https://github.com/diyism/colab_kaldi2/raw/refs/heads/main/icefall_egs_wenetspeech_ASR_prepare.sh
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/diyism/colab_kaldi2/refs/heads/main/icefall_egs_wenetspeech_ASR_prepare.sh [following]
--2024-10-04 18:56:19--  https://raw.githubusercontent.com/diyism/colab_kaldi2/refs/heads/main/icefall_egs_wenetspeech_ASR_prepare.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1912 (1.9K) [text/plain]
Saving to: ‘prepare.sh’

prepare.sh          100%[===================>]   1.87K  --.-KB/s    in 0s      

2024-10-04 18:56:19 (32.8 MB/s)

In [ ]:
#5.training

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/wenetspeech/ASR && \
  ./zipformer/train.py

2024-10-04 18:40:20.271096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 18:40:20.304396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-04 18:40:20.314901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-04 18:40:20.340132: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-04 18:40:21.832160: W tensorflow/compiler/tf2

In [ ]:
#6. decode a single sound file
!pip install kaldifeat==1.25.5.dev20240914+cuda12.1.torch2.4.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/wenetspeech/ASR && \
  ./zipformer/pretrained.py \
    --checkpoint /content/icefall/egs/yesno/ASR/tdnn/exp/epoch-14.pt \
    --words-file /content/icefall/egs/yesno/ASR/data/lang_phone/words.txt \
    --HLG /content/icefall/egs/yesno/ASR/data/lang_phone/HLG.pt \
    /content/icefall/egs/yesno/ASR/download/waves_yesno/0_0_1_0_1_0_0_1.wav

Looking in links: https://csukuangfj.github.io/kaldifeat/cuda.html
2024-09-23 16:50:45,387 INFO [pretrained.py:136] {'feature_dim': 23, 'num_classes': 4, 'sample_rate': 8000, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': '/content/icefall/egs/yesno/ASR/tdnn/exp/epoch-14.pt', 'words_file': '/content/icefall/egs/yesno/ASR/data/lang_phone/words.txt', 'HLG': '/content/icefall/egs/yesno/ASR/data/lang_phone/HLG.pt', 'sound_files': ['/content/icefall/egs/yesno/ASR/download/waves_yesno/0_0_1_0_1_0_0_1.wav']}
2024-09-23 16:50:45,407 INFO [pretrained.py:142] device: cuda:0
2024-09-23 16:50:45,407 INFO [pretrained.py:144] Creating model
/content/icefall/egs/yesno/ASR/./tdnn/pretrained.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary

In [ ]:
#7. decode all waves
! export PYTHONPATH=/content/icefall:$PYTHONPATH && \
  cd /content/icefall/egs/yesno/ASR && \
  ./tdnn/decode.py

! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/recogs-test_set.txt

2024-09-23 16:51:54,314 INFO [decode.py:262] Decoding started
2024-09-23 16:51:54,314 INFO [decode.py:263] {'exp_dir': PosixPath('tdnn/exp'), 'lang_dir': PosixPath('data/lang_phone'), 'feature_dim': 23, 'search_beam': 20, 'output_beam': 8, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'epoch': 14, 'avg': 2, 'export': False, 'feature_dir': PosixPath('data/fbank'), 'max_duration': 30.0, 'bucketing_sampler': False, 'num_buckets': 10, 'concatenate_cuts': False, 'duration_factor': 1.0, 'gap': 1.0, 'on_the_fly_feats': False, 'shuffle': False, 'return_cuts': True, 'num_workers': 2, 'env_info': {'k2-version': '1.24.4', 'k2-build-type': 'Release', 'k2-with-cuda': True, 'k2-git-sha1': 'cf664841c6d93e21e59b40aade84869b76c919c1', 'k2-git-date': 'Thu Sep 5 19:25:17 2024', 'lhotse-version': '1.28.0.dev+git.bc2c0a2.clean', 'torch-version': '2.4.1+cu121', 'torch-cuda-available': True, 'torch-cuda-version': '12.1', 'python-version': '3.10', 'icefall-git-branch': 'maste

In [ ]:
#8. show the detailed WER
! cd /content/icefall/egs/yesno/ASR && \
  cat tdnn/exp/errs-test_set.txt

%WER = 0.42
Errors: 0 insertions, 1 deletions, 0 substitutions, over 240 reference words (239 correct)
Search below for sections starting with PER-UTT DETAILS:, SUBSTITUTIONS:, DELETIONS:, INSERTIONS:, PER-WORD STATS:

PER-UTT DETAILS: corr or (ref->hyp)  
0_0_0_1_0_0_0_1-0:	NO NO NO YES NO NO NO YES
0_0_1_0_0_0_1_0-1:	NO NO YES NO NO NO YES NO
0_0_1_0_0_1_1_1-2:	NO NO YES NO NO YES YES YES
0_0_1_0_1_0_0_1-3:	NO NO YES NO YES NO NO YES
0_0_1_1_0_0_0_1-4:	NO NO YES YES NO NO NO YES
0_0_1_1_0_1_1_0-5:	NO NO YES YES NO YES YES NO
0_0_1_1_1_0_0_0-6:	NO NO YES YES YES NO NO NO
0_0_1_1_1_1_0_0-7:	NO NO YES YES YES YES NO NO
0_1_0_0_0_1_0_0-8:	NO YES NO NO NO YES NO NO
0_1_0_0_1_0_1_0-9:	NO YES NO NO YES NO YES NO
0_1_0_1_0_0_0_0-10:	NO YES NO YES NO NO NO (NO->*)
0_1_0_1_1_1_0_0-11:	NO YES NO YES YES YES NO NO
0_1_1_0_0_1_1_1-12:	NO YES YES NO NO YES YES YES
0_1_1_1_0_0_1_0-13:	NO YES YES YES NO NO YES NO
0_1_1_1_1_0_1_0-14:	NO YES YES YES YES NO YES NO
1_0_0_0_0_0_0_0-15:	YES NO NO NO NO NO